In [1]:
import os
import tkinter as tk
from dotenv import load_dotenv
from sqlalchemy.orm import scoped_session, Session
import sys
sys.path.append('..')
from src.repo.orm import OpenPool, PrayerRequestORM
load_dotenv()
pg_uri = os.environ.get('PRAYERS_PG_DATABASE_URL')

class PrayerLabelerApp:
    def __init__(self, master: tk.Tk):
        self.master = master
        self.master.title("Prayer Labeler")
        self.master.geometry("600x400")

        self.prayer_text = tk.StringVar()
        self.prayer_text.set("No prayer request available")

        self.label_text = tk.Label(master, textvariable=self.prayer_text, wraplength=400)
        self.label_text.pack()

        self.button_frame = tk.Frame(master)
        self.button_frame.pack(side=tk.BOTTOM, anchor=tk.SE, padx=10, pady=10)

        self.prayer_button = tk.Button(self.button_frame, text="Prayer Request", command=self.label_prayer, padx=10,
                                       background='orange', foreground='white')
        self.prayer_button.pack(side=tk.RIGHT, padx=15, pady=5)

        self.praise_button = tk.Button(self.button_frame, text="Praise", command=self.label_praise, padx=10,
                                       background='green', foreground='white')
        self.praise_button.pack(side=tk.RIGHT, padx=15, pady=5)

        self.id_at = 0

        # Connect to the database
        self.pool: scoped_session[Session] = OpenPool(pg_uri)

        # Load initial prayer request
        self.load_prayer_request()

    def load_prayer_request(self):
        # Query the database for an unlabeled prayer request
        with self.pool() as session:
            prayer_request = session.query(PrayerRequestORM).filter(PrayerRequestORM.prayer_type == None).first()
            if prayer_request == None:
                print("No more prayer requests")
                self.master.quit()
                return
            self.id_at = prayer_request.id
            if prayer_request:
                self.prayer_text.set(prayer_request.request)
            else:
                self.prayer_text.set("No more prayer requests")

    def label_prayer(self):
        self.update_label("Prayer Request")

    def label_praise(self):
        self.update_label("Praise")

    def update_label(self, label):
        # Update the label in the database
        with self.pool() as session:
            prayer_request = session.query(PrayerRequestORM).filter(PrayerRequestORM.id == self.id_at).first()
            if prayer_request:
                prayer_request.prayer_type = label
                session.commit()
            # Load the next prayer request
        self.load_prayer_request()

root = tk.Tk()
app = PrayerLabelerApp(root)
root.mainloop()

No more prayer requests


: 